In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 6.60 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-06 19:19:51.223555: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 19:19:53.241118: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 19:19:53.241260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 19:19:53.518657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 19:19:54.141267: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 19:19:54.142510: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [8]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [9]:
tester = Tester(run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[30, 21], [21, 12], [33, 21]],
            regularizers=[0.02, 0.2, 0.5],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[30, 21], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  0.8530908750008166
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.16537034199973277
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.16107021600055305
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.16249568700004602
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.16024814200045512
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.2247972249997474
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 +++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_3_9,0.913750,0.710421,0.936656,0.685013,0.775012,0.361975,1.936414,0.054267,0.508876,0.281572
model_1_3_8,0.912813,0.709470,0.940142,0.689340,0.778998,0.365903,1.942774,0.051280,0.501886,0.276583
model_1_3_7,0.911847,0.708500,0.943436,0.693707,0.782944,0.369961,1.949264,0.048459,0.494830,0.271644
model_1_3_6,0.910848,0.707509,0.946528,0.698114,0.786847,0.374150,1.955886,0.045809,0.487710,0.266760
model_1_3_5,0.909818,0.706498,0.949412,0.702558,0.790702,0.378476,1.962647,0.043339,0.480531,0.261935
model_1_3_4,0.908754,0.705466,0.952079,0.707038,0.794507,0.382942,1.969551,0.041054,0.473293,0.257174
model_1_3_3,0.907655,0.704411,0.954521,0.711550,0.798255,0.387552,1.976603,0.038961,0.466004,0.252483
model_1_3_2,0.906521,0.703334,0.956730,0.716091,0.801942,0.392312,1.983810,0.037069,0.458667,0.247868
model_1_3_1,0.905350,0.702232,0.958697,0.720658,0.805563,0.397225,1.991178,0.035384,0.451290,0.243337
model_1_3_0,0.904142,0.701105,0.960411,0.725245,0.809110,0.402297,1.998714,0.033916,0.443879,0.238897


DataFrame salvo em ../results/metrics_1_3
+++++++++++ [1_4] | 1 ++++++++++++++++++
Elapsed time:  0.9518979619997481
Stopped at epoch:  40
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 2 ++++++++++++++++++
Elapsed time:  0.16651582200029225
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_4] | 3 ++++++++++++++++++
Elapsed time:  0.15948560800006817
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 4 ++++++++++++++++++
Elapsed time:  0.24804741400021157
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 5 ++++++++++++++++++
Elapsed time:  0.16977482400034205
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 6 ++++++++++++++++++
Elapsed time:  0.1990700360001938
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_4] | 7 ++++++++++++++++++
Elapsed time:  0.1730476

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_4_24,0.937301,0.751682,0.663903,0.978145,0.899003,0.263136,1.660502,1.168915,0.095406,0.632161
model_1_4_23,0.937412,0.751453,0.667105,0.978034,0.899854,0.262667,1.662032,1.157781,0.095891,0.626836
model_1_4_22,0.937518,0.751217,0.670316,0.977913,0.900704,0.262225,1.663613,1.146610,0.096419,0.621515
model_1_4_21,0.937616,0.750973,0.673538,0.977782,0.901553,0.261811,1.665245,1.135406,0.096994,0.616200
model_1_4_20,0.937708,0.750721,0.676768,0.977639,0.902401,0.261426,1.666931,1.124172,0.097615,0.610894
model_1_4_19,0.937793,0.750460,0.680006,0.977486,0.903247,0.261071,1.668672,1.112912,0.098285,0.605599
model_1_4_18,0.937870,0.750192,0.683250,0.977321,0.904090,0.260747,1.670470,1.101630,0.099006,0.600318
model_1_4_17,0.937939,0.749914,0.686498,0.977143,0.904931,0.260458,1.672327,1.090333,0.099779,0.595056
model_1_4_16,0.937999,0.749627,0.689750,0.976954,0.905768,0.260203,1.674245,1.079022,0.100607,0.589814
model_1_4_15,0.938051,0.749331,0.693004,0.976752,0.906602,0.259985,1.676225,1.067706,0.101490,0.584598


DataFrame salvo em ../results/metrics_1_4
+++++++++++ [1_5] | 1 ++++++++++++++++++
Elapsed time:  0.17332083000019338
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_5] | 2 ++++++++++++++++++
Elapsed time:  0.15863004099992395
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_5] | 3 ++++++++++++++++++
Elapsed time:  0.15808941399973264
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_5] | 4 ++++++++++++++++++
Elapsed time:  0.18107775000044057
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_5] | 5 ++++++++++++++++++
Elapsed time:  0.15983251499983453
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_5] | 6 ++++++++++++++++++
Elapsed time:  0.15895252300015272
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_5] | 7 ++++++++++++++++++
Elapsed time:  0.164177

...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_3_9,0.970724,0.771027,0.443099,0.991267,0.892196,0.122864,1.531140,0.576923,0.037308,0.307115
model_8_3_10,0.969760,0.770995,0.394684,0.990272,0.882647,0.126912,1.531357,0.627078,0.041558,0.334318
model_8_3_8,0.971568,0.770931,0.494827,0.992308,0.902382,0.119323,1.531785,0.523335,0.032857,0.278096
model_8_3_11,0.968727,0.770868,0.349644,0.989330,0.873752,0.131244,1.532207,0.673738,0.045580,0.359659
model_8_3_12,0.967666,0.770673,0.307946,0.988446,0.865508,0.135698,1.533510,0.716935,0.049356,0.383146
model_8_3_7,0.972222,0.770663,0.549668,0.993387,0.913162,0.116580,1.533576,0.466522,0.028251,0.247387
model_8_3_13,0.966605,0.770432,0.269493,0.987622,0.857898,0.140153,1.535123,0.756770,0.052879,0.404824
model_8_3_6,0.972593,0.770171,0.607234,0.994487,0.924454,0.115022,1.536866,0.406887,0.023551,0.215219
model_8_3_14,0.965563,0.770161,0.234149,0.986857,0.850898,0.144523,1.536936,0.793385,0.056148,0.424767
model_8_3_15,0.964557,0.769873,0.201747,0.986150,0.844477,0.148746,1.538862,0.826952,0.059168,0.443060


DataFrame salvo em ../results/metrics_8_3
+++++++++++ [8_4] | 1 ++++++++++++++++++
Elapsed time:  0.5174353580005118
Stopped at epoch:  21
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_4] | 2 ++++++++++++++++++
Elapsed time:  0.16002288100025908
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_4] | 3 ++++++++++++++++++
Elapsed time:  0.1999760669996249
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_4] | 4 ++++++++++++++++++
Elapsed time:  0.15642388200012647
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_4] | 5 ++++++++++++++++++
Elapsed time:  0.16350945999965916
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_4] | 6 ++++++++++++++++++
Elapsed time:  0.1570605799997793
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_4] | 7 ++++++++++++++++++
Elapsed time:  0.17336593

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_4_0,0.968930,0.732849,0.824756,0.987138,0.944782,0.130395,1.786441,0.715917,0.076883,0.396400
model_8_4_1,0.968378,0.731776,0.820275,0.987074,0.943480,0.132711,1.793614,0.734221,0.077270,0.405746
model_8_4_2,0.967840,0.730794,0.816151,0.986961,0.942260,0.134969,1.800179,0.751068,0.077944,0.414506
model_8_4_3,0.967322,0.729897,0.812363,0.986814,0.941121,0.137143,1.806180,0.766544,0.078823,0.422683
model_8_4_4,0.966827,0.729078,0.808889,0.986644,0.940061,0.139219,1.811657,0.780737,0.079841,0.430289
model_8_4_5,0.966359,0.728331,0.805708,0.986459,0.939079,0.141185,1.816650,0.793733,0.080944,0.437338
model_8_4_6,0.965918,0.727651,0.802800,0.986267,0.938172,0.143034,1.821196,0.805614,0.082091,0.443852
model_8_4_7,0.965506,0.727033,0.800145,0.986073,0.937336,0.144765,1.825331,0.816460,0.083251,0.449855
model_8_4_8,0.965122,0.726471,0.797724,0.985881,0.936567,0.146377,1.829088,0.826349,0.084399,0.455374
model_8_4_9,0.964766,0.725961,0.795520,0.985694,0.935862,0.147871,1.832498,0.835353,0.085518,0.460436


DataFrame salvo em ../results/metrics_8_4
+++++++++++ [8_5] | 1 ++++++++++++++++++


: 